In [41]:
import json
import csv
#import pandas as pd
import numpy as np
from sys import argv

In [48]:
file = open('test_link_time.json',"r")
link_time = json.load(file)
#link_time.keys()
#link_time['link1_2']

In [12]:
def get_graph():
    #filename = argv[1]
    filename = 'test.txt'  # single-direction graph
    graph = {}
    with open(filename) as g:
        i = 1
        for line in g:
            l = line.split()
            vertex = int(l.pop(0))
            graph[vertex] = {}
            for x in l:
                adj_vert, link = x.split(",")#map(str, x.split(","))
                graph[vertex][adj_vert] = link
            #print ("Got graph. Ex: line ",i, ":", graph[i])
            i += 1
    return graph

In [13]:
G = get_graph()
#G.keys()

In [124]:
def dijkstra_score_d(G, shortest_distances, v, w):
    time = int(np.floor(shortest_distances[v]))
    name = 'link'+ str(v) + '_' + str(w)
    return shortest_distances[v] + float(link_time[name][time])

In [125]:
# dijkstra based on time-dependent-cost
def dijkstra_d(G, source):
    unprocessed = set(G.keys()) # vertices whose shortest paths from source have not yet been calculated
    unprocessed.remove(source)
    shortest_distances = {source: 0}
    shortest_route = {source: source}
 
    for i in range(len(G) - 1):
        # find a vertex with the next shortest path, i.e. minimal Dijkstra score
        m, closest_head = float('inf'), 0  #shortest distance, vertex index
        #print(shortest_distances)
        for tail in shortest_distances:
            for head in G[tail]:
                head = int(head)
                if head in unprocessed:
                    d = dijkstra_score_d(G, shortest_distances, tail, head)
                    if d < m:
                        m, closest_head = d, head
                        closest_nb = tail
        try:
            #print(closest_head)
            unprocessed.remove(closest_head)
            shortest_distances[closest_head] = m
            shortest_route[closest_head] = closest_nb
        except:
            break
 
    # in case G is not fully connected
    for vertex in unprocessed:
        shortest_distances[vertex] = float('inf')
 
    return [shortest_distances,shortest_route]

In [131]:
def dijkstra_score_s(G, shortest_distances, v, w, t):
    name = 'link'+ str(v) + '_' + str(w)
    return shortest_distances[v] + float(link_time[name][t])

In [156]:
# dijkstra based on static cost at time t
def dijkstra_s(G, source, t):
    unprocessed = set(G.keys()) # vertices whose shortest paths from source have not yet been calculated
    unprocessed.remove(source)
    shortest_distances = {source: 0}
    shortest_route = {source: source}
 
    for i in range(len(G) - 1):
        # find a vertex with the next shortest path, i.e. minimal Dijkstra score
        m, closest_head = float('inf'), 0  #shortest distance, vertex index
        #print(shortest_distances)
        for tail in shortest_distances:
            for head in G[tail]:
                head = int(head)
                if head in unprocessed:
                    d = dijkstra_score_s(G, shortest_distances, tail, head, t)
                    if d < m:
                        m, closest_head = d, head
                        closest_nb = tail
        try:
            #print(closest_head,m,closest_nb)
            unprocessed.remove(closest_head)
            shortest_distances[closest_head] = m
            shortest_route[closest_head] = closest_nb
        except:
            break
 
    # in case G is not fully connected
    for vertex in unprocessed:
        shortest_distances[vertex] = float('inf')
 
    return [shortest_distances,shortest_route]

In [161]:
# test for multi-step static (greedy)
o = 2
d = 10
time = 0
[dis,rt] = dijkstra_s(G,o,time)
#print(dis)
#print(rt)
# recover the whole route between O and D
print('ETA:',dis[d])
if d in rt:
    route = [d]
    D = d
    while rt[D] != o:
        D = rt[D]
        route.append(D)
    route.append(o)
    # convers vector route
    R = []
    for i in range(len(route)):
        R.append(route[len(route)-1-i])
    print('Estimated route:',R)
else:
    print('Destination cannot arrive!')


ETA: 5.0
Estimated route: [2, 6, 8, 10]


In [159]:
# update data after passing a link
ATA = 0
A_route = [o]
while o != d:
    new = route[len(route) - 2]
    name = 'link'+ str(o) + '_' + str(new)
    #print(name)
    o = new
    A_route.append(o)
    ATA += link_time[name][time]
    time = int(np.floor(ATA))
    [dis,rt] = dijkstra_s(G,o,time)
    #print(dis)
    #print(rt)
    # recover the whole route between O and D
    if d in rt:
        route = [d]
        D = d
        while rt[D] != o:
            D = rt[D]
            route.append(D)
        route.append(o)
        #print(route)
    else:
        print('Destination cannot arrive!')
print('ATA:',ATA)
print('Actual route:',A_route)

link2_6
link6_9
link9_10
ATA: 8
Actual route: [2, 6, 9, 10]


In [162]:
# test for dynamic
o = 2
[dis,rt] = dijkstra_d(G,o)
print('ETA:',dis[d])
#print(dis)
#print(rt)

# recover the whole route between O and D
d = 10
if d in rt:
    route = [d]
    D = d
    while rt[D] != o:
        D = rt[D]
        route.append(D)
    route.append(o)
    # convers vector route
    R = []
    for i in range(len(route)):
        R.append(route[len(route)-1-i])
    print('Estimated route:',R)
else:
    print('Destination cannot arrive!')

ETA: 5.0
Estimated route: [2, 5, 7, 10]
